<span style="color: blue">Author:A Nikfarjma</span>

## <span style="color: yellow">Data Source and description</span>

**Source:** 
Data has been Collected from NCBI's(The National Center for Biotechnology Information) GEO(Gene Expression Omnibus) DataSet.

https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE62944

NCBI GEO Accession Number: GSE62944

This data is consist of RNA-Seq data for 9264 tumor samples and 741 normal samples across 24 cancer types from The Cancer Genome Atlas with "Rsubread"

Lets Take a look!

There are two text file in the data set that contain the sample name for cancerous and normal samples with their respective ids.



Also data has been normalize using RNA sequincing methis FPKM and TPM. For building a classification model, the choice between raw counts, FPKM, or TPM depends on the specific requirements of our analysis and the algorithms we plan to use. Raw counts are often preferred for differential expression analysis using tools like DESeq2 or edgeR, as these methods have their own normalization procedures. However, since our classification model or downstream analysis requires normalized expression values, TPM is generally favored over FPKM because TPM accounts for sequencing depth and gene length, facilitating more accurate comparisons across samples. It's crucial to ensure that the data format and normalization method align with the assumptions of your chosen analytical tools.

In [1]:
#normal smaples
with open("./data/NormalSample.txt", "r") as file:
    data = [line.strip().split("\t") for line in file]

# Print first 3 rows
print(data[:3])

[['TCGA-K4-A3WV-11A-21R-A22U-07', 'BLCA'], ['TCGA-49-6742-11A-01R-1858-07', 'LUAD'], ['TCGA-DD-A3A2-11A-11R-A213-07', 'LIHC']]


<span style="color: green">Now lets map these genes</span>

In [2]:
#lets extract unique values of the genes
unique_genes=set()
for sample in data:
    if sample[1] not in unique_genes:
        unique_genes.add(sample[1])
unique_genes


{'BLCA',
 'BRCA',
 'CESC',
 'CHOL',
 'COAD',
 'ESCA',
 'GBM',
 'HNSC',
 'KICH',
 'KIRC',
 'KIRP',
 'LIHC',
 'LUAD',
 'LUSC',
 'PAAD',
 'PCPG',
 'PRAD',
 'READ',
 'SARC',
 'SKCM',
 'STAD',
 'THCA',
 'THYM',
 'UCEC'}

it seams like the only gene related to breast cancer here is **BRCA**

In [3]:
#extracting all the BRCA samples

#healthy_brca_samples=set()
healthy_genes_sample_dic={gene:[] for gene in unique_genes}
# Populate dictionary
for sample_id, cancer_type in data:
    if cancer_type in healthy_genes_sample_dic:
        healthy_genes_sample_dic[cancer_type].append(sample_id)



In [4]:
healthy_genes_sample_dic

{'PRAD': ['TCGA-EJ-7781-11A-01R-2118-07',
  'TCGA-CH-5769-11A-01R-1580-07',
  'TCGA-EJ-7317-11A-01R-2118-07',
  'TCGA-G9-6496-11A-01R-1789-07',
  'TCGA-G9-6365-11A-01R-1789-07',
  'TCGA-EJ-7314-11A-01R-2118-07',
  'TCGA-HC-7747-11A-01R-2118-07',
  'TCGA-EJ-7123-11A-01R-1965-07',
  'TCGA-EJ-7315-11A-01R-2118-07',
  'TCGA-CH-5761-11A-01R-1580-07',
  'TCGA-EJ-7789-11A-01R-2118-07',
  'TCGA-EJ-7115-11A-01R-2118-07',
  'TCGA-G9-6356-11A-01R-1789-07',
  'TCGA-EJ-A8FO-11A-11R-A36G-07',
  'TCGA-EJ-7783-11A-01R-2118-07',
  'TCGA-CH-5768-11A-01R-1580-07',
  'TCGA-EJ-7327-11A-01R-2118-07',
  'TCGA-EJ-7792-11A-01R-2118-07',
  'TCGA-EJ-7794-11A-01R-2118-07',
  'TCGA-HC-7740-11A-01R-2118-07',
  'TCGA-EJ-7793-11A-01R-2263-07',
  'TCGA-EJ-7785-11A-01R-2118-07',
  'TCGA-G9-6333-11A-01R-1965-07',
  'TCGA-HC-8259-11A-01R-2263-07',
  'TCGA-G9-6351-11A-01R-1965-07',
  'TCGA-HC-7745-11A-01R-2118-07',
  'TCGA-EJ-7125-11A-01R-1965-07',
  'TCGA-EJ-7331-11A-01R-2118-07',
  'TCGA-G9-6384-11A-01R-1858-07',
  'TCG

now we maped all the samples for healthy genes now lets look at the cancerous ones

In [5]:
with open("./data/CancerousSaples.txt", "r") as file:
    data = [line.strip().split("\t") for line in file]
cUnique_genes=set()
for sample in data:
    if sample[1] not in cUnique_genes:
        cUnique_genes.add(sample[1])

cancerous_genes_sample_dic={gene:[] for gene in unique_genes}
# Populate dictionary
for sample_id, cancer_type in data:
    if cancer_type in cancerous_genes_sample_dic:
        cancerous_genes_sample_dic[cancer_type].append(sample_id)
cancerous_genes_sample_dic

{'PRAD': ['TCGA-G9-A9S0-01A-11R-A41O-07',
  'TCGA-XK-AAIW-01A-11R-A41O-07',
  'TCGA-J4-A67M-01A-11R-A30B-07',
  'TCGA-J4-A67Q-01A-21R-A30B-07',
  'TCGA-XJ-A83G-01A-11R-A352-07',
  'TCGA-H9-A6BY-01A-11R-A30B-07',
  'TCGA-KK-A59Z-01A-12R-A26U-07',
  'TCGA-EJ-A65M-01A-11R-A29R-07',
  'TCGA-CH-5748-01A-11R-1580-07',
  'TCGA-HC-A9TE-01A-11R-A41O-07',
  'TCGA-J9-A52C-01A-11R-A26U-07',
  'TCGA-G9-6371-01A-11R-1789-07',
  'TCGA-EJ-A46E-01A-31R-A250-07',
  'TCGA-CH-5764-01A-21R-1580-07',
  'TCGA-M7-A721-01A-12R-A32O-07',
  'TCGA-KK-A6DY-01A-12R-A311-07',
  'TCGA-ZG-A9LU-01A-11R-A41O-07',
  'TCGA-FC-7961-01A-11R-A29R-07',
  'TCGA-XJ-A9DK-01A-11R-A37L-07',
  'TCGA-VN-A88Q-01A-11R-A352-07',
  'TCGA-KK-A7AP-01A-12R-A33R-07',
  'TCGA-HC-A48F-01A-11R-A250-07',
  'TCGA-HC-8265-01B-04R-2302-07',
  'TCGA-ZG-A9LZ-01A-11R-A41O-07',
  'TCGA-V1-A9ZG-01A-11R-A41O-07',
  'TCGA-G9-6362-01A-11R-1789-07',
  'TCGA-G9-A9S7-01A-11R-A41O-07',
  'TCGA-J4-A83N-01A-11R-A352-07',
  'TCGA-KC-A4BR-01A-32R-A32Y-07',
  'TCG

Now we have referances to samples and genes lets extract all the BRCA samples

In [6]:
# lets look at GSM1536837_01_27_15_TCGA_20.Illumina.tumor_Rsubread_TPM
with open('data/GSE62944_RAW/GSM1536837_06_01_15_TCGA_24.tumor_Rsubread_TPM.txt','r') as file:
    feature_data=[line.strip().split("\t") for line in file]

feature_data[0][-3:-1]

['TCGA-ZS-A9CF-02A-11R-A38B-07', 'TCGA-ZS-A9CG-01A-11R-A37K-07']

## <span style="color: yellow">

In [7]:
len(data)

9264

In [8]:
import pandas as pd

df = pd.DataFrame(feature_data)

df.head()

,0,1,2,3,4,5,6,7,8,9,...,9255,9256,9257,9258,9259,9260,9261,9262,9263,9264
0,TCGA-02-0047-01A-01R-1849-01,TCGA-02-0055-01A-01R-1849-01,TCGA-02-2483-01A-01R-1849-01,TCGA-02-2485-01A-01R-1849-01,TCGA-02-2486-01A-01R-1849-01,TCGA-04-1331-01A-01R-1569-13,TCGA-04-1332-01A-01R-1564-13,TCGA-04-1337-01A-01R-1564-13,TCGA-04-1338-01A-01R-1564-13,TCGA-04-1341-01A-01R-1564-13,...,TCGA-ZP-A9D2-01A-11R-A38B-07,TCGA-ZP-A9D4-01A-11R-A37K-07,TCGA-ZQ-A9CR-01A-11R-A39E-31,TCGA-ZS-A9CD-01A-11R-A37K-07,TCGA-ZS-A9CE-01A-11R-A37K-07,TCGA-ZS-A9CF-01A-11R-A38B-07,TCGA-ZS-A9CF-02A-11R-A38B-07,TCGA-ZS-A9CG-01A-11R-A37K-07,TCGA-ZX-AA5X-01A-11R-A42T-07,None
1,1/2-SBSRNA4,2.34180779897709,1.64575215578575,2.43263380829254,2.04036406475485,1.40859926165477,4.08500123669713,1.64344250568849,4.17885803869778,1.73244853368479,...,0.347388825555485,0.733966333074115,0.853099951263001,0.814110159868275,0.236654111917491,0.456498091772548,2.0007951391706,4.18063767545344,0.509342962675824,1.48192576535246
2,A1BG,11.7173931223676,24.7763572132346,20.2648719245462,5.1177123833555,5.95979586986965,22.4929513168985,4.26866974722146,9.32833631054866,7.55995847739945,...,2322.58593459571,43.9997958639196,1793.07061325774,4.85285257810494,2033.96723633235,3907.83790754622,544.860528291095,233.078057214062,580.658764687128,5.78202791256206
3,A1BG-AS1,1.33274150888264,1.08603706301478,1.09297209133143,0.707189363771972,0.608536246570792,3.29101508083205,0.636544632484977,1.93088255678914,0.696446879034295,...,7.02361491668872,0.453430961753005,9.41200210705638,0.388637270859652,8.52748094151365,9.08861178822967,4.04526960884422,1.95248795513533,2.99579748187921,1.50919876535332
4,A1CF,0.00783961263378836,0.015495311808889,0.00479039331850954,0.00585469442145148,0.00544099991686763,0.0235780321144235,0.0284571322739637,0.0190765373788806,0.0366295297288906,...,48.594490263584,37.4965936190835,59.8263832981506,2.09513329984465,27.4178826337677,40.4449662284777,68.4793019038917,96.5849604190457,60.2122597568698,0.00992201320489947


need to drop the first row and set the second one as header and shift them one column over

In [ ]:
# Creating the DataFrame
moded_df = pd.DataFrame(feature_data)

# Extract first row as column names and shift right
new_columns = [""] + moded_df.iloc[0, :-1].tolist()  # Shift column names one position right

# Drop the first row since it's now the header
moded_df = moded_df.iloc[1:].reset_index(drop=True)

# Assign the new column names
moded_df.columns = new_columns

moded_df.head()

,,TCGA-02-0047-01A-01R-1849-01,TCGA-02-0055-01A-01R-1849-01,TCGA-02-2483-01A-01R-1849-01,TCGA-02-2485-01A-01R-1849-01,TCGA-02-2486-01A-01R-1849-01,TCGA-04-1331-01A-01R-1569-13,TCGA-04-1332-01A-01R-1564-13,TCGA-04-1337-01A-01R-1564-13,TCGA-04-1338-01A-01R-1564-13,...,TCGA-ZP-A9D1-01A-11R-A38B-07,TCGA-ZP-A9D2-01A-11R-A38B-07,TCGA-ZP-A9D4-01A-11R-A37K-07,TCGA-ZQ-A9CR-01A-11R-A39E-31,TCGA-ZS-A9CD-01A-11R-A37K-07,TCGA-ZS-A9CE-01A-11R-A37K-07,TCGA-ZS-A9CF-01A-11R-A38B-07,TCGA-ZS-A9CF-02A-11R-A38B-07,TCGA-ZS-A9CG-01A-11R-A37K-07,TCGA-ZX-AA5X-01A-11R-A42T-07
0,1/2-SBSRNA4,2.34180779897709,1.64575215578575,2.43263380829254,2.04036406475485,1.40859926165477,4.08500123669713,1.64344250568849,4.17885803869778,1.73244853368479,...,0.347388825555485,0.733966333074115,0.853099951263001,0.814110159868275,0.236654111917491,0.456498091772548,2.0007951391706,4.18063767545344,0.509342962675824,1.48192576535246
1,A1BG,11.7173931223676,24.7763572132346,20.2648719245462,5.1177123833555,5.95979586986965,22.4929513168985,4.26866974722146,9.32833631054866,7.55995847739945,...,2322.58593459571,43.9997958639196,1793.07061325774,4.85285257810494,2033.96723633235,3907.83790754622,544.860528291095,233.078057214062,580.658764687128,5.78202791256206
2,A1BG-AS1,1.33274150888264,1.08603706301478,1.09297209133143,0.707189363771972,0.608536246570792,3.29101508083205,0.636544632484977,1.93088255678914,0.696446879034295,...,7.02361491668872,0.453430961753005,9.41200210705638,0.388637270859652,8.52748094151365,9.08861178822967,4.04526960884422,1.95248795513533,2.99579748187921,1.50919876535332
3,A1CF,0.00783961263378836,0.015495311808889,0.00479039331850954,0.00585469442145148,0.00544099991686763,0.0235780321144235,0.0284571322739637,0.0190765373788806,0.0366295297288906,...,48.594490263584,37.4965936190835,59.8263832981506,2.09513329984465,27.4178826337677,40.4449662284777,68.4793019038917,96.5849604190457,60.2122597568698,0.00992201320489947
4,A2LD1,3.03652529001364,3.10263327586671,1.43091177120877,6.13049025914624,6.05450592575091,3.52917089274419,2.97042467356097,3.58488583814053,3.78740883457729,...,5.46423253889593,10.4344641043496,3.85912874382051,2.34831379648535,5.89794546122663,9.18921577523948,10.4562840780572,12.9114147880195,7.08272708139775,3.0288867847545


In [10]:
moded_df=moded_df.set_index(moded_df.columns[0])
moded_df.head()

,TCGA-02-0047-01A-01R-1849-01,TCGA-02-0055-01A-01R-1849-01,TCGA-02-2483-01A-01R-1849-01,TCGA-02-2485-01A-01R-1849-01,TCGA-02-2486-01A-01R-1849-01,TCGA-04-1331-01A-01R-1569-13,TCGA-04-1332-01A-01R-1564-13,TCGA-04-1337-01A-01R-1564-13,TCGA-04-1338-01A-01R-1564-13,TCGA-04-1341-01A-01R-1564-13,...,TCGA-ZP-A9D1-01A-11R-A38B-07,TCGA-ZP-A9D2-01A-11R-A38B-07,TCGA-ZP-A9D4-01A-11R-A37K-07,TCGA-ZQ-A9CR-01A-11R-A39E-31,TCGA-ZS-A9CD-01A-11R-A37K-07,TCGA-ZS-A9CE-01A-11R-A37K-07,TCGA-ZS-A9CF-01A-11R-A38B-07,TCGA-ZS-A9CF-02A-11R-A38B-07,TCGA-ZS-A9CG-01A-11R-A37K-07,TCGA-ZX-AA5X-01A-11R-A42T-07
,,,,,,,,,,,,,,,,,,,,,
1/2-SBSRNA4,2.34180779897709,1.64575215578575,2.43263380829254,2.04036406475485,1.40859926165477,4.08500123669713,1.64344250568849,4.17885803869778,1.73244853368479,0.948406127013232,...,0.347388825555485,0.733966333074115,0.853099951263001,0.814110159868275,0.236654111917491,0.456498091772548,2.0007951391706,4.18063767545344,0.509342962675824,1.48192576535246
A1BG,11.7173931223676,24.7763572132346,20.2648719245462,5.1177123833555,5.95979586986965,22.4929513168985,4.26866974722146,9.32833631054866,7.55995847739945,14.1505845997296,...,2322.58593459571,43.9997958639196,1793.07061325774,4.85285257810494,2033.96723633235,3907.83790754622,544.860528291095,233.078057214062,580.658764687128,5.78202791256206
A1BG-AS1,1.33274150888264,1.08603706301478,1.09297209133143,0.707189363771972,0.608536246570792,3.29101508083205,0.636544632484977,1.93088255678914,0.696446879034295,0.435584084750929,...,7.02361491668872,0.453430961753005,9.41200210705638,0.388637270859652,8.52748094151365,9.08861178822967,4.04526960884422,1.95248795513533,2.99579748187921,1.50919876535332
A1CF,0.00783961263378836,0.015495311808889,0.00479039331850954,0.00585469442145148,0.00544099991686763,0.0235780321144235,0.0284571322739637,0.0190765373788806,0.0366295297288906,0.0126998238857055,...,48.594490263584,37.4965936190835,59.8263832981506,2.09513329984465,27.4178826337677,40.4449662284777,68.4793019038917,96.5849604190457,60.2122597568698,0.00992201320489947
A2LD1,3.03652529001364,3.10263327586671,1.43091177120877,6.13049025914624,6.05450592575091,3.52917089274419,2.97042467356097,3.58488583814053,3.78740883457729,1.73694772371811,...,5.46423253889593,10.4344641043496,3.85912874382051,2.34831379648535,5.89794546122663,9.18921577523948,10.4562840780572,12.9114147880195,7.08272708139775,3.0288867847545


Now we have all the samples and genes that showed up in cancerous data

In [11]:
moded_df.index

Index(['1/2-SBSRNA4', 'A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1',
       'A2MP1', 'A4GALT', 'A4GNT',
       ...
       'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX', 'ZZEF1',
       'ZZZ3', 'tAKR'],
      dtype='object', name='', length=23368)

<span style="color: green">the indecies are feature id</span>

Now that we have all the samples seperated in healthy and non healthy genes sample we can alsi extract dates using enterez api. and get dates of sample esxtraction and use for analysis like the evolution of cancer. Meaning How genes matation changes as time passes. 

In [2]:
from Bio import Entrez
import ssl
import Bio
import pandas as pd
# Disable SSL verification (for development purposes)
ssl._create_default_https_context = ssl._create_unverified_context
Entrez.email = "ashkan.nikfarjam@sjsu.edu"



In [15]:
import xml.etree.ElementTree as ET
# List to store row-wise structured data
times_series_sample_healthy = []

# Loop through each gene and its associated samples
for gene, samples in healthy_genes_sample_dic.items():
    for sample in samples:
        # Search for the sample in BioSample database
        handle = Entrez.esearch(db="biosample", term=sample)
        record = Entrez.read(handle)
        handle.close()

        collection_date = "Not found"  # Default value

        if record["IdList"]:
            biosample_id = record["IdList"][0]

            # Fetch metadata
            handle = Entrez.efetch(db="biosample", id=biosample_id, rettype="xml")
            biosample_data = handle.read()
            handle.close()

            # Parse XML to extract collection date
            root = ET.fromstring(biosample_data)

            for attribute in root.iter("Attribute"):
                if "harmonized_name" in attribute.attrib and attribute.attrib["harmonized_name"] == "collection date":
                    collection_date = attribute.text
                    break
        
        # Append structured data to list
        times_series_sample_healthy.append({
            "Gene": gene,
            "Sample": sample,
            "Collection_Date": collection_date
        })

# Convert list of dictionaries to DataFrame
df_date_healthy = pd.DataFrame(times_series_sample_healthy)

df_date_healthy.head()

,Gene,Sample,Collection_Date
0,PRAD,TCGA-EJ-7781-11A-01R-2118-07,Not found
1,PRAD,TCGA-CH-5769-11A-01R-1580-07,Not found
2,PRAD,TCGA-EJ-7317-11A-01R-2118-07,Not found
3,PRAD,TCGA-G9-6496-11A-01R-1789-07,Not found
4,PRAD,TCGA-G9-6365-11A-01R-1789-07,Not found


In [16]:
df_date_healthy[df_date_healthy['Gene']=='BRCA'].shape

(113, 3)

In [17]:
cancerous_genes_sample_dic['BRCA']

['TCGA-A2-A0T1-01A-21R-A084-07',
 'TCGA-A7-A3J0-01A-11R-A213-07',
 'TCGA-AO-A126-01A-11R-A10J-07',
 'TCGA-E2-A14X-01A-11R-A115-07',
 'TCGA-A7-A13G-01A-11R-A13Q-07',
 'TCGA-AR-A0TP-01A-11R-A084-07',
 'TCGA-E2-A15S-01A-11R-A115-07',
 'TCGA-A2-A1FW-01A-11R-A13Q-07',
 'TCGA-HN-A2OB-01A-21R-A27Q-07',
 'TCGA-B6-A0I1-01A-11R-A21T-07',
 'TCGA-LL-A440-01A-11R-A24H-07',
 'TCGA-E2-A14T-01A-11R-A115-07',
 'TCGA-LL-A740-01A-21R-A32P-07',
 'TCGA-BH-A5IZ-01A-11R-A27Q-07',
 'TCGA-E9-A1N5-01A-11R-A14D-07',
 'TCGA-BH-A18L-01A-32R-A12D-07',
 'TCGA-GM-A2DC-01A-11R-A18M-07',
 'TCGA-E9-A1NC-01A-21R-A26B-07',
 'TCGA-BH-A0C0-01A-21R-A056-07',
 'TCGA-A2-A0EU-01A-22R-A056-07',
 'TCGA-AR-A24U-01A-11R-A169-07',
 'TCGA-A8-A096-01A-11R-A00Z-07',
 'TCGA-EW-A1PG-01A-11R-A144-07',
 'TCGA-D8-A1XJ-01A-11R-A14M-07',
 'TCGA-A8-A082-01A-11R-A00Z-07',
 'TCGA-A2-A3KD-01A-12R-A213-07',
 'TCGA-GM-A2DK-01A-21R-A180-07',
 'TCGA-B6-A0WY-01A-11R-A109-07',
 'TCGA-E2-A159-01A-11R-A115-07',
 'TCGA-A8-A06P-01A-11R-A00Z-07',
 'TCGA-BH-

In [18]:
BRCA_mutated_data=moded_df.loc[:,cancerous_genes_sample_dic['BRCA']]
BRCA_mutated_data

,TCGA-A2-A0T1-01A-21R-A084-07,TCGA-A7-A3J0-01A-11R-A213-07,TCGA-AO-A126-01A-11R-A10J-07,TCGA-E2-A14X-01A-11R-A115-07,TCGA-A7-A13G-01A-11R-A13Q-07,TCGA-AR-A0TP-01A-11R-A084-07,TCGA-E2-A15S-01A-11R-A115-07,TCGA-A2-A1FW-01A-11R-A13Q-07,TCGA-HN-A2OB-01A-21R-A27Q-07,TCGA-B6-A0I1-01A-11R-A21T-07,...,TCGA-A2-A0YJ-01A-11R-A109-07,TCGA-A2-A0T4-01A-31R-A084-07,TCGA-AR-A1AH-01A-11R-A12D-07,TCGA-E9-A1RG-01A-11R-A14D-07,TCGA-GM-A2DL-01A-11R-A18M-07,TCGA-A8-A09R-01A-11R-A00Z-07,TCGA-AC-A5XU-01A-11R-A28M-07,TCGA-BH-A0C7-01B-11R-A115-07,TCGA-BH-A1EW-01A-11R-A137-07,TCGA-A7-A0DC-01A-11R-A00Z-07
,,,,,,,,,,,,,,,,,,,,,
1/2-SBSRNA4,0.960894001732585,1.79708641615056,2.48136842272879,1.25877918887833,2.31111187622,2.23687134793837,2.76660213716546,2.28447385971219,2.21287365182981,2.97309145249474,...,0.763348471245634,1.05799128383335,4.93549009636928,2.61136572692273,0.696066154236605,0.764936645643112,3.01945635331633,2.35876124201148,3.02597926104386,1.44938113627721
A1BG,3.55441830878723,24.8511023337201,37.6048986305345,3.14831871353749,4.08870963418863,11.8845011062784,14.7424175604732,1.26272158177772,19.9403509036443,2.64817262444747,...,3.39792574029033,17.7731752080883,11.4623783098821,27.5701664657272,16.1253026532216,13.7482851647991,40.6012177755066,8.61226150638332,8.47222789141078,2.25670952477252
A1BG-AS1,1.12624073442509,3.72311870425808,1.53058196884434,1.78370238476053,1.72553127179168,2.11927465153879,2.14703912334954,0.862179740068278,3.0963311769728,0.90588561158173,...,1.15139674339897,1.95732770240254,2.09078085490862,0.970986692827605,1.72006770085652,3.13133220293153,4.80568571334337,2.27592411781154,2.56805720385617,1.10062110487387
A1CF,0.00839155127969162,0,0.0295666343487768,0.0486228954429215,0.341331734325332,0.0146510597853031,0.0126295721446844,0.0504746695795784,0,0.0892333041676456,...,0,0.00625025561986329,0.0283241474729796,0.00904345356159619,0.0291275452830145,0.0298755296064832,0.00513974242788515,0.034951133773597,0.0900444149926421,0.0533042963479159
A2LD1,4.55869505679023,6.19852777232165,5.50420560541654,6.27771883565517,2.52092239644118,4.61679472085385,2.01753245758846,2.33460165332695,5.50246643755226,1.47579220910952,...,2.87593912497247,5.31370982522715,0.960721648796729,1.92951411789502,3.00215993822184,8.37758895373583,6.15792664219459,21.2242957078167,7.29682927216734,1.14403175472163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,10.87191636006,12.2433532160665,9.13528384909196,18.6954742347446,29.5773531241006,19.0673376256323,21.8463708185679,29.1821081372339,16.9286814513,14.4721854012189,...,17.6177476112723,11.4552814990698,42.2983815304041,5.04328425116633,13.1789850309646,17.727185981507,7.56123719397127,20.0753158576823,24.3106780708509,13.7774322426059
ZYX,365.273345093633,152.190474694495,254.570884887423,227.541749180432,9.47016319225069,644.173691635008,60.2257399811681,77.4656969413474,329.061927512053,161.564591054938,...,211.999508369364,175.708445244203,195.954842285442,172.331469201123,394.519695387113,243.0103963273,334.687618478921,161.233598165552,169.315645320559,3.9444486433987
ZZEF1,12.1084963137152,27.4519209528469,5.88806299340222,16.2859268370279,23.4029471173933,8.80908759202247,20.6017777750802,26.6416404065616,16.3279364652754,7.37311025502842,...,21.4353734317249,8.74956033628851,9.13091346113153,7.75159681554729,9.54336018367033,24.7583823015003,19.7066903602046,22.0896270690066,21.9590996783082,12.2650917130979


This is our cancerous BRCA gene expresion lets save it to CSV and move on to creat a data seth for healthy genes!

In [21]:
BRCA_mutated_data.to_csv("./data/mutatedDataSet.CSV")

In [3]:
with open('./data/GSE62944_RAW/GSM1697009_06_01_15_TCGA_24.normal_Rsubread_TPM.txt') as file:
    healthy_genes=[line.strip().split("\t") for line in file]
healthy_df=pd.DataFrame(healthy_genes)
# Extract first row as column names and shift right
new_columns = [""] + healthy_df.iloc[0, :-1].tolist()  # Shift column names one position right

# Drop the first row since it's now the header
healthy_df = healthy_df.iloc[1:].reset_index(drop=True)

# Assign the new column names
healthy_df.columns = new_columns

healthy_df.head()


,,TCGA-06-0675-11A-32R-A36H-07,TCGA-06-0678-11A-32R-A36H-07,TCGA-06-0680-11A-32R-A36H-07,TCGA-06-0681-11A-41R-A36H-07,TCGA-06-AABW-11A-31R-A36H-07,TCGA-22-4593-11A-01R-1820-07,TCGA-22-4609-11A-01R-2125-07,TCGA-22-5471-11A-01R-1635-07,TCGA-22-5472-11A-11R-1635-07,...,TCGA-W5-AA2X-11A-11R-A41I-07,TCGA-W5-AA30-11A-11R-A41I-07,TCGA-W5-AA31-11A-11R-A41I-07,TCGA-W5-AA34-11A-11R-A41I-07,TCGA-WA-A7GZ-11A-11R-A34R-07,TCGA-X7-A8D6-11A-22R-A42C-07,TCGA-X7-A8D7-11A-11R-A42C-07,TCGA-Y8-A8RY-11A-11R-A37K-07,TCGA-YB-A89D-11A-11R-A36G-07,TCGA-ZU-A8S4-11A-11R-A41I-07
0,1/2-SBSRNA4,3.18073422220701,2.13049012484436,2.1761167256058,1.71464925560361,1.55566888090254,0.778035365333389,1.06999077162721,1.14273389244002,0.881162891368868,...,0.915108840158311,0.293867620204237,0.0871453640505864,0.403677389434792,0.69081426677416,3.0160903056496,6.26075626091982,3.78428864815151,1.32437679958846,0.227014015554126
1,A1BG,7.39732335355717,5.1722413033141,7.30475724374385,11.9796934330644,21.0022192919542,3.30495200094336,4.70554984193677,2.99717228862688,2.43868352302866,...,2262.64939429268,2361.61579445944,1548.36672875698,1721.08220718153,1.20945154798853,10.5106241014704,30.7325961736698,2.96214515362105,8.78444282548321,2028.49291424024
2,A1BG-AS1,2.59022643394693,1.68470757139805,3.31338461373735,3.30164856662506,2.09686636482216,0.879558199010695,1.1833657417953,1.25275824298537,0.890781218704234,...,14.2141996463866,12.1690857458612,10.3758216197413,9.67309575434355,0.358130689763203,2.68938531241141,4.74297181156493,0.812348785394276,0.761647232123888,11.8723853802107
3,A1CF,0.0532403304063405,0.017117255111563,0.0303538861837658,0.00615009571388339,0.013585759891076,0.00252059026815952,0.00413305604933622,0.0183624109151449,0.00316055082619975,...,61.7073902545322,42.7892256575703,75.5037804880722,67.4564305895944,0.0213472800698393,0.00932020769798,0.0137686732893755,0.0971257068299844,0.845927786570591,47.3217659969279
4,A2LD1,3.01460303845154,2.7812473427231,2.53074454312974,2.22062308864465,3.21082563973426,4.09550768491891,3.13388868161999,2.53150787862123,3.16419259075836,...,4.95607024278445,3.66514570820693,9.17867627631877,16.5161945614391,2.64520780223995,4.72666002042746,4.41406527935657,7.80396013712328,3.89440685052667,5.55292078404725


In [4]:
healthy_df=healthy_df.set_index(healthy_df.columns[0])
healthy_df=healthy_df.loc[:,healthy_genes_sample_dic['BRCA']]
healthy_df.head()

NameError: name 'healthy_genes_sample_dic' is not defined

In [27]:
healthy_df.to_csv("./data/normalDataset.csv")

In [31]:
healthy_df.shape

(23368, 113)

In [1]:
import plotly.graph_objects as go
import plotly.express as px
import numpy as np

# Simulating data (Use your actual healthy_df)
# healthy_df = pd.read_csv("your_data.csv")  # Uncomment to load real data

# Convert DataFrame to long format
# melted_df = healthy_df.stack().reset_index()
# melted_df.columns = ["Gene", "Sample", "Expression Level"]

# Subset only 4 samples for speed
df_subset = healthy_df.iloc[:, :4]

# Convert to long format
melted_df = df_subset.stack().reset_index()
melted_df.columns = ["Gene", "Sample", "Expression Level"]

# Create scatter plot
fig1 = px.scatter(
    melted_df,
    x="Gene",
    y="Sample",
    color="Expression Level",
    color_continuous_scale="Viridis",
    opacity=0.7,  # Adjust opacity for visibility
    title="Gene Expression Scatter Plot (Subset of 4 Samples)"
)

# Turn off axis labels for better readability
fig1.update_layout(
    xaxis_title="Genes",
    yaxis_title="Samples",
    xaxis=dict(showticklabels=False),
    yaxis=dict(showticklabels=False),
    coloraxis_colorbar=dict(title="Expression Level"),
    width=1000, height=800  # Adjust size
)

# Show plot
fig1.show()

NameError: name 'healthy_df' is not defined